In [ ]:
%load_ext autoreload
%autoreload 2
from colocate import ui, erddap_query, run, plot

import hvplot
import hvplot.pandas

import panel as pn
import param
pn.extension()

In [ ]:
ui.m

In [ ]:
ui.dates

In [ ]:
ui.drpdwn

In [ ]:
ui.kw

In [ ]:
ui.button

In [ ]:
ui.df

In [ ]:
all_coords = erddap_query.get_coordinates(ui.df, **ui.kw)

In [ ]:
all_coords

In [ ]:
all_coords.columns

In [ ]:
figure = plot.plot_datashader(all_coords)

In [ ]:
figure

In [ ]:
# list type is necessary to add an empty value, otherwise could use numpy.ndarrays directly in ObjectSelector:
server_list = ui.df['server'].unique().tolist()
server_list[:0] = [None]
institution_list = ui.df['Institution'].unique().tolist()
institution_list[:0] = [None]
dataset_id_list = servers = ui.df['Dataset ID'].unique().tolist()
dataset_id_list[:0] = [None]

class ErddapParams(param.Parameterized):
    #sel_server = param.ObjectSelector(objects=servers)
    #sel_institution = param.ObjectSelector(objects=institutions)
    #sel_datatypes = param.ObjectSelector(objects=datatypes)
    #sel_server = param.ObjectSelector(objects=servers, default="", check_on_set=False, allow_None=True, label="Select an ERDDAP Server:")
    #sel_institution = param.ObjectSelector(objects=instiutions, default="", check_on_set=False, allow_None=True, label="Select an Institution:")
    
    sel_server = param.ObjectSelector(objects=server_list, default=None, allow_None=True, label="Select an ERDDAP Server:")
    sel_institution = param.ObjectSelector(objects=institution_list, default=None, allow_None=True, label="Select an Institution:")

#erddap_params = ErddapParams(servers=servers.tolist(), institutions=institutions.tolist())
erddap_params = ErddapParams()

In [ ]:
class ChartParams(param.Parameterized):
    sel_datasetid = param.ObjectSelector(objects=ui.df['Dataset ID'].unique().tolist(), default=None, allow_None=True, label="Select Dataset to Plot:")
    
chart_params = ChartParams()

In [ ]:
filtered_df = ui.df

@pn.depends(erddap_params.param.sel_server, erddap_params.param.sel_institution)
def filter_df(server, institution):
    
    if server is not None:
        try:
            filter &= ui.df.server.str.contains(server)
        except NameError:
            filter = ui.df.server.str.contains(server) 
            
    if institution is not None:
        try:
            filter &= ui.df.Institution.str.contains(institution)
        except NameError:
            filter = ui.df.Institution.str.contains(institution)
    
    #filter &=  ui.df.server=server if server is not None
    #filter &=  ui.df.institution=institution if institution is not None
    try:
        global filtered_df
        filtered_df = ui.df[filter].drop(columns=['Summary'])
        
        # filter the server_list according to the 'Institution' value selected:
        if institution is not None:
            server_list = ui.df[ui.df.Institution.str.contains(institution)].server.unique().tolist()
        else:
            server_list = ui.df.server.unique().tolist()
        server_list[:0] = [None]
        erddap_params.param['sel_server'].objects = server_list
        

        # filter the institution_list according to the 'server' value selected:
        if server is not None:
            institution_list = ui.df[ui.df.server.str.contains(server)].Institution.unique().tolist()
        else:
            institution_list = ui.df.Institution.unique().tolist()
        institution_list[:0] = [None]
        erddap_params.param['sel_institution'].objects = institution_list
        
        
        # set the chart_params pulldown to correspond to only the Dataset ID values present in the filtered dataframe (ie apply the same filter used on ui.df):
        dataset_filter = ui.df[filter]['Dataset ID'].unique().tolist()
        chart_params.param['sel_datasetid'].objects = dataset_filter
        
        # To Do: add a function to plot only the dataset selected in the chart_params selector in a DataShader plot
        
        return ui.df[filter].drop(columns=['Summary']).head(10)
    except Exception: 
        return ui.df.drop(columns=['Summary']).head(10)

In [ ]:
#gspec = pn.GridSpec(sizing_mode='stretch_both', max_height=1000)
gspec = pn.GridSpec(sizing_mode='stretch_both', height_policy='fit', width_policy='auto', height=1600)
gspec[0, 0] = erddap_params
gspec[0, 1:4] = pn.panel(filter_df, width=800, max_height=300)
gspec[1, 0] = chart_params
#gspec[2, :] = pn.Spacer(background='purple', margin=0)
gspec[2, :] = figure

gspec

### To Do: add a function to plot only the dataset selected in the chart_params selector in a DataShader plot

```
@pn.depends(chart_params.param.sel_datasetid)
def plot_dataset(datasetid):

    some hvPlot/Datashated plotting code here....

```


Debugging/this notebook is a WIP....

In [ ]:
filtered_df